In [ ]:
import sys
import skvideo.io
import skvideo.datasets

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution3D, MaxPooling3D

In [ ]:
from keras.optimizers import SGD, RMSprop
from keras.utils import np_utils, generic_utils

In [ ]:
import tensorflow as tf
import os
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import cv2
from sklearn.cross_validation import train_test_split
from sklearn import cross_validation
from sklearn import preprocessing

In [ ]:
hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))

### Image specification

In [ ]:
img_rows,img_cols,img_depth=200,300,15

### Training data

In [ ]:
X_tr = [] # variable to store entire dataset

In [ ]:
len(X_tr)

### Open files

In [ ]:
# load class 0
filename = '/home/misha/Documents/Thesis/Dataset/BNG_white_1'
videodata = skvideo.io.vreader(filename + '.mov')
frames = open(filename + '.csv', 'r')

my_frame = 0
frame_number = 0

for frame in videodata:
    if (frame_number >= 10 and frame_number < 510):
        
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        res_img = cv2.resize(frame, (img_cols, img_rows), interpolation=cv2.INTER_AREA)
        X_tr.append(res_img)
#         print(frame_number)
    frame_number += 1
print(len(X_tr))



# load class 1
filename = '/home/misha/Documents/Thesis/Dataset/race_night'
videodata = skvideo.io.vreader(filename + '.mov')
frames = open(filename + '.csv', 'r')

my_frame = 0
frame_number = 0

for frame in videodata:
    if (frame_number >= 10 and frame_number < 510):
        
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        res_img = cv2.resize(frame, (img_cols, img_rows), interpolation=cv2.INTER_AREA)
        X_tr.append(res_img)
#         print(frame_number)
    frame_number += 1
print(len(X_tr))

### Reading accident class training

In [ ]:
plt.imshow(X_tr[-1],cmap='gray')
plt.show()


##### Image resize

### Reading all frames classes (takes long time)

### CNN Training parameters

In [ ]:
# Importaint step
X_tr_array = np.array(X_tr)


In [ ]:

X_tr_array = np.expand_dims(X_tr_array, axis=4)
print(X_tr_array.shape)

##### Assign label to each class

In [ ]:
num_samples = len(X_tr_array) # 397 - 0, 515 - 1, 610 - 0, 
# train_set = np.zeros(num_samples, 1, img_rows,img_cols,img_depth))
label = np.ones((num_samples,),dtype = int)
label[0:500] = 0
label[500:] = 1

train_data = [X_tr_array,label]

(X_train, y_train) = (train_data[0],train_data[1])
print('X_Train shape:', X_train.shape)

In [ ]:
train_set = np.zeros((num_samples, img_rows,img_cols,img_depth))
patch_size = 15



In [ ]:
print(train_set.shape, 'train samples')

##### CNN training parameters

In [ ]:
batch_size = 2
nb_classes = 2 
nb_epoch = 10


##### convert class vectors to binary class matrices

In [ ]:
Y_train = np_utils.to_categorical(y_train, nb_classes)

##### number of convolutional filters to use at each layer

In [ ]:
nb_filters = [32, 32]

##### level of pooling to perform at each layer (POOL x POOL)

In [ ]:
nb_pool = [2, 2]

##### Level of convolution to perform at each layer (CONV x CONV)

In [ ]:
nb_conv = [3,3]

### Pre-processing

In [ ]:
train_set = train_set.astype('float32')

train_set -= np.mean(train_set)

train_set /=np.max(train_set)

In [ ]:
print(img_rows, img_cols, img_depth)

### Define model

In [ ]:
model = Sequential()
model.add(Convolution3D(16, (3, 3, 2),
input_shape=(img_rows, img_cols, img_depth, 1), activation='relu'))

model.add(MaxPooling3D(pool_size=(nb_pool[0], nb_pool[0], nb_pool[0])))
# model.add(Dropout(0.5))

model.add(Convolution3D(32, (3, 3, 3), activation='relu'))

model.add(MaxPooling3D(pool_size=(2, 2, 1)))
# model.add(Dropout(0.5))

model.add(Convolution3D(64, (3, 3, 2), activation='relu'))
# model.add(Dropout(0.5))

model.add(Convolution3D(128, (3, 3, 2), activation='relu'))

model.add(Convolution3D(256, (2, 2, 2), activation='relu'))

model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(128, init='normal', activation='relu'))

# model.add(Dropout(0.5))

model.add(Dense(nb_classes,init='normal'))

model.add(Activation('softmax'))

# model.optimizer.lr.assign(0.1)

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='Adam')


In [ ]:
model.summary()

In [ ]:
# Split the data
X_train_new, X_val_new, y_train_new,y_val_new =  train_test_split(train_set, Y_train, test_size=0.2, random_state=4)

In [ ]:
X_train_new = np.expand_dims(X_train_new, axis=4)
print(X_train_new.shape)

X_val_new = np.expand_dims(X_val_new, axis=4)
print(X_val_new.shape)

#### Training

In [ ]:
hist = model.fit(X_train_new, y_train_new, validation_data=(X_val_new,y_val_new),
          batch_size=batch_size,nb_epoch = nb_epoch,shuffle=True)

### Split the data

### Train the model

### Evaluate the model

In [ ]:
score = model.evaluate(X_val_new, y_val_new, batch_size=batch_size)
print('Test score:', score[0])
print('Test accuracy:', score[1]) 


In [ ]:
train_loss=hist.history['loss']
val_loss=hist.history['val_loss']
train_acc=hist.history['acc']
val_acc=hist.history['val_acc']

plt.plot(train_loss)
plt.plot(val_loss)
plt.plot(train_acc)
plt.plot(val_acc)
plt.xlabel('num of Epochs')
plt.grid(True)
plt.legend(['train_loss','val_loss','train_acc', 'val_acc'])

plt.show()

### Plots the result